In [1]:
#%% Transitive Inference - Model batch analysis  #########################

import sys

import task_train_test as tk
import nn_analyses     as nna
import behavior_functions as bf
import ti_functions as ti


#%% Select batch #######################################################################

# specify batch directory here #
jobname         = 'ti351'            # specify name of batch (cluster) job
currentdir      = %pwd               # directory containing model_files/
paramsdir       = '%s/model_files/%s/' % (currentdir,jobname)
localdir        = '%s/model_files/' % currentdir


########################################################################################

In [2]:
# Load parameters from the init (parameter) file for this batch ###
sys.path.insert(0, paramsdir)
from init import singleout, params, genfilename
p        = singleout()
tf       = tk.Get_taskfunc(p)

In [3]:
#%% Select models to analyze and print performance ######################################

# select which models to analyze #
only_perfect   = 1      # 0: all models, 1: perfect on train+test, 2: perfect on train only
TT_test        = [0]    # 0: correct on full delay duration, 13: correct shortened delay

if p.Model == 8:     # LR (logistic regression)
   Model_select   = -1    # -4: early stop, -1: lowest loss
else:                # MLP and RNN (multi-layer perceptron and recurrent neural network)
   Model_select   = -4    # -4: early stop, -1: lowest loss

Perfs          = bf.read_model_performance(jobname, localdir, TT_test, Model_select, verbose=True, only_perfect=only_perfect )


(/Users/kendrick/emergent_transitive/model_files/ti351/) Getting all models' performance
TID: 0 || Model: -1 Model_variant: 0 actfn: 1 Delay: 20 input_train: 0 output_train: 0 jit: 0 Seed: 0 || Train: 100 Test: 100 || Delay: 30  || (ep: 585)
TID: 1 || Model: -1 Model_variant: 0 actfn: 1 Delay: 20 input_train: 0 output_train: 0 jit: 0 Seed: 1 || Train: 100 Test: 100 || Delay: 35  || (ep: 514)
TID: 2 || Model: -1 Model_variant: 0 actfn: 1 Delay: 20 input_train: 0 output_train: 0 jit: 0 Seed: 2 || Train: 100 Test: 100 || Delay: 20  || (ep: 513)
TID: 3 || Model: -1 Model_variant: 0 actfn: 1 Delay: 20 input_train: 0 output_train: 0 jit: 0 Seed: 3 || Train: 100 Test: 100 || Delay: 15  || (ep: 617)
TID: 4 || Model: -1 Model_variant: 0 actfn: 1 Delay: 20 input_train: 0 output_train: 0 jit: 0 Seed: 4 || Train: 100 Test: 100 || Delay: 30  || (ep: 603)
TID: 5 || Model: -1 Model_variant: 0 actfn: 1 Delay: 20 input_train: 0 output_train: 0 jit: 0 Seed: 5 || Train: 100 Test: 100 || Delay: 25  || (ep

In [4]:
#%% Obtain behavior (simulation) data across models #########################

MAX_INSTANCES  = 200  # maximum number of model instances to select

Behaviors      = ti.Behavior_across_models(jobname,  Perfs['TIDs'], Model_select, localdir, MAX_INSTANCES )


(/Users/kendrick/emergent_transitive/model_files/ti351/) Plotting selected TIDs Error Pattern
TID: 0 | noise: 0.85, test: 80.1267, train: 62.8167, no_response: 17.3714
TID: 1 | noise: 0.95, test: 77.6733, train: 61.9333, no_response: 19.119
TID: 2 | noise: 0.8, test: 82.2867, train: 64.3333, no_response: 17.4143
TID: 3 | noise: 0.75, test: 81.8867, train: 64.2833, no_response: 16.319
TID: 4 | noise: 0.8, test: 81.1467, train: 64.6667, no_response: 16.4
TID: 5 | noise: 0.85, test: 80.6667, train: 63.9, no_response: 17.4095
TID: 6 | noise: 0.9, test: 79.0667, train: 63.3833, no_response: 17.5286
TID: 7 | noise: 0.75, test: 79.4667, train: 63.4333, no_response: 17.2
TID: 8 | noise: 0.8, test: 81.4467, train: 63.4667, no_response: 16.4476
TID: 9 | noise: 0.8, test: 81.04, train: 62.9, no_response: 18.4048
TID: 10 | noise: 0.85, test: 79.8467, train: 62.7333, no_response: 17.2619
TID: 11 | noise: 0.85, test: 78.2533, train: 63.45, no_response: 18.0571
TID: 12 | noise: 0.8, test: 80.4133, tr

In [5]:
#%% Plot Models' Task Behavior ######## # 

import matplotlib as mpl 
mpl.use("Qt5Agg") # set the backend  

# Plot individual models #
tf.Behavior_plot( p, Behaviors, TIDs_plot=[5] ) # f-RNN simple
# tf.Behavior_plot( p, Behaviors, TIDs_plot=[10] ) # f-RNN complex
# tf.Behavior_plot( p, Behaviors, TIDs_plot=[15] ) # r-RNN simple **
# tf.Behavior_plot( p, Behaviors, TIDs_plot=[10] ) # r-RNN complex

# Plot all instances a model variant #
if 0:
    Variants_to_plot = [0,4]     # (constraint regime) 0: highest, 1: high, 2: intermediate, 3: low, 4: lowest
    bf.Behavior_Plot_Variants( p, tf, params, Perfs['TIDs_dims'], Behaviors , 'Model_variant', Variants_to_plot, -1)


/Users/kendrick/emergent_transitive/ti_functions.py:1340: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(YTICKLABELS)
/Users/kendrick/emergent_transitive/ti_functions.py:1340: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(YTICKLABELS)


RNN Max RT: 0.461828 milliseconds


In [6]:
#%% Calculate (or load) analyses ###################

#### Select analysis settings ####
Calculate_neurals = 0      # 0: load file if available, 1: calculate
topPCs            = 10     # Specify how many PCs to retain for analysis
PCA_setting       = 0      # 0: standard PCA, 1: delay PCA
######################################

# Calculate (or load) analysis file #
if p.T > 1:

   if PCA_setting == 0:  
      PCA_geometry      = 0   # 0: Default PCA (full trial)  1: Delay PCA 
      topPCs_geometry   = 10  
   elif PCA_setting == 1: 
      PCA_geometry      = 1   # 0: Default PCA (full trial)  1: Delay PCA 
      topPCs_geometry   = 2    

   filename = '%s%s_PCA_%g_localproc' % (localdir,jobname,PCA_geometry)

   if Calculate_neurals or 'Neurals' not in locals():
      out = []
      if Calculate_neurals == 0:
         out = tk.readfile(filename)
         if len(out) > 0:
            Behaviors, Neurals = (out[0], out[1])
      if len(out) == 0:
         Neurals = ti.Calculate_Neurals( jobname,  Behaviors['TIDs'], Model_select, localdir, topPCs, PCA_geometry, topPCs_geometry, MAX_INSTANCES )
         tk.writefile( filename, [Behaviors, Neurals] )  

   tk.readfile(filename, extension = ".p", silent=False)

Reading file (pickle) /Users/kendrick/emergent_transitive/model_files/ti351_PCA_0_localproc.p...
Reading file (pickle) /Users/kendrick/emergent_transitive/model_files/ti351_PCA_0_localproc.p...


In [7]:
####### End-order behavior ###############

# Histogram of end order index #
ti.Plot_End_Order(p, Behaviors, Neurals)

# PCA variance explained #
ti.Plot_PCA_varexp(p, Neurals, topPCs = 3)

/Users/kendrick/emergent_transitive/ti_functions.py:4331: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  fig.tight_layout()


Variant 0, varexp, mean ± s.d.: 87.2 ± 1.9
Variant 1, varexp, mean ± s.d.: 88.6 ± 2.2
Variant 2, varexp, mean ± s.d.: 85.5 ± 2.6
Variant 3, varexp, mean ± s.d.: 79.0 ± 4.2
Variant 4, varexp, mean ± s.d.: 91.4 ± 1.7


/Users/kendrick/emergent_transitive/ti_functions.py:4364: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  fig.tight_layout()


In [8]:
####### Geometry during delay period ########

# PCA trajectories #
ti.Plot_Delay_PCA(p, Neurals, Behaviors, jobname, TIDs_plot = [5,6,19,24,127,189,191,194] )  # f-RNN simple
# ti.Plot_Delay_PCA(p, Neurals, Behaviors, jobname, TIDs_plot = [655,626,716,734,752,787,867,887,897,826,922,940,941,830,878] )  # f-RNN complex

# Geometric indices, histograms#
ti.Plot_GI(p, Neurals, Behaviors, normalize = 1, random_data=True)        # geometric indexes

# Scatter of end-order behavior vs. geometric indices #
# ti.Plot_End_vs_GI(p, Behaviors, Neurals)


In [9]:
####### Linear Dynamics ######

# R-squareds of linear dynamics #
ti.Plot_LD_R2(p, Neurals )      

# Eigenvalues (pooled for each constraint regime) #
for var in range(5):  # (constraint regime) 0: highest, 1: high, 2: intermediate, 3: low, 4: lowest
   nna.plot_eig(p, Neurals, Variant_plot=[var])     # collective eigenspectra

R2, variant 0: mean ± s.d.: 0.81 ± 0.02
R2, variant 1: mean ± s.d.: 0.64 ± 0.05
R2, variant 2: mean ± s.d.: 0.66 ± 0.04
R2, variant 3: mean ± s.d.: 0.71 ± 0.06
R2, variant 4: mean ± s.d.: 0.61 ± 0.10


/Users/kendrick/emergent_transitive/ti_functions.py:4180: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  fig.tight_layout()
/Users/kendrick/emergent_transitive/nn_analyses.py:647: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  fig.tight_layout()


In [10]:
###### Readout analyses ##########

ti.Plot_Readout_Example(p, Neurals, jobname, TID = 22 )   # ex. 1, f-RNN simple
# ti.Plot_Readout_Example(p, Neurals, jobname, TID = 956 )  # ex. 2. f-RNN complex
# ti.Plot_Readout_Example(p, Neurals, jobname, TID = 50 )   # ex. 3. r-RNN simple
# ti.Plot_Readout_Example(p, Neurals, jobname, TID = 913 )    # ex. 4. r-RNN complex
# ti.Plot_Readout_Example(p, Neurals, jobname, TID = 895 )    # ex. 5. r-RNN complex

# Changes in readout representation, histogram #
ti.Plot_Readout_Study(p, Neurals, Behaviors)  

# Scatter of End-order behavior vs. Encoding index #
ti.Plot_End_vs_ERI(p, Behaviors, Neurals)    


[[ 8.86301622e-01  8.77349935e-01  7.05476671e-01  8.35734665e-01
   7.17737823e-01  1.00644659e+00  9.30298015e-01]
 [ 1.07705816e-01  9.87541300e-02 -7.31191340e-02  5.71388600e-02
  -6.08579817e-02  2.27850789e-01  1.51702210e-01]
 [ 3.26615420e-02  2.37098557e-02 -1.48163408e-01 -1.79054144e-02
  -1.35902256e-01  1.52806515e-01  7.66579357e-02]
 [ 8.32886544e-02  7.43369681e-02 -9.75362959e-02  3.27216980e-02
  -8.52751437e-02  2.03433627e-01  1.27285048e-01]
 [-1.26308354e-02 -2.15825218e-02 -1.93455786e-01 -6.31977918e-02
  -1.81194634e-01  1.07514137e-01  3.13655582e-02]
 [ 1.68844208e-01  1.59892522e-01 -1.19807425e-02  1.18277251e-01
   2.80409785e-04  2.88989180e-01  2.12840602e-01]
 [-1.02140498e+00 -1.03035667e+00 -1.20222993e+00 -1.07197194e+00
  -1.18996878e+00 -9.01260009e-01 -9.77408588e-01]]
[[ 1.2030042   1.53328457  1.29140288  1.47860466  1.52124181  1.54151313
   1.86556103]
 [-0.40219724 -0.07191687 -0.31379857 -0.12659679 -0.08395964 -0.06368832
   0.26035958]
 [

In [11]:
###### Angles between axes ##########
ti.Plot_Angles(p, Neurals)  # angles


/Users/kendrick/emergent_transitive/ti_functions.py:4281: UserWarning: This figure was using constrained_layout, but that is incompatible with subplots_adjust and/or tight_layout; disabling constrained_layout.
  fig.tight_layout()


In [12]:
###### (Single model instance) plot changes in geometric indices over delay ##########

TID = 0

ti.Plot_Neural_Traces_Single_RNN(p, Neurals, TID)


NameError: name 'M' is not defined